In [6]:
import yfinance as yf
import quantstats as qs
import pandas as pd
import warnings

# Ignore all warnings
warnings.filterwarnings('ignore')

# Set the ticker symbol and time period
ticker_symbol = "MNQ=F"
#start_date = "2024-03-01"
#end_date = "2024-03-11"

# Fetch intraday data using yfinance for the last 10 days with 15-minute intervals
data= yf.download(ticker_symbol, period="max")  # Downloads maximum available data

# Calculate Pivot Points for 15-minute intervals
data['Pivot_Point'] = (data['High'] + data['Low'] + data['Close']) / 3
data['Support_1'] = 2 * data['Pivot_Point'] - data['High']
data['Resistance_1'] = 2 * data['Pivot_Point'] - data['Low']
data['Support_2'] = data['Pivot_Point'] - (data['High'] - data['Low'])
data['Resistance_2'] = data['Pivot_Point'] + (data['High'] - data['Low'])
data['Support_3'] = data['Low'] - 2 * (data['High'] - data['Pivot_Point'])
data['Resistance_3'] = data['High'] + 2 * (data['Pivot_Point'] - data['Low'])

# Calculate 10 period moving average for the close price
data['10_Day_MA'] = data['Close'].rolling(window=10).mean()

# Round numbers
data = data.round(2)

# Create a Signal column with 1, whenever the pivot is greater than current open// 
data.loc[data['Pivot_Point'] > data['Open'], 'Signal'] = 1

# Calculate daily range
data['Range'] = data['High'] - data['Low']

# Calculate daily change
data['Change'] = data['Close'] - data['Open']

# Calculate profit // MNQ is 2 dollars per point // Position is opened on Open, and closed on same day close, this is why we use CHANGE 
# for calculating the profit. 
data['Profit'] = data.apply(lambda row: row['Change'] * 2 if row['Signal'] == 1 else 0, axis=1)

profit = data['Profit']

# Initial Balance, Cumulative Profit, Balance
initial_balance = 10000
data['cumulative_profit'] = data['Profit'].cumsum()
data['Balance'] = initial_balance + data['cumulative_profit']

# Calculate 'returns' column
data['returns'] = data['Profit'] / data['Balance']

# Display the data
display(data)

# Display performance metrics using quantstats
#qs.reports.full(data['returns'])
#Quanstats#  Load returns, without having to convert to a series 
returns_series = data['returns']

#

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume,Pivot_Point,Support_1,Resistance_1,Support_2,...,Support_3,Resistance_3,10_Day_MA,Signal,Range,Change,Profit,cumulative_profit,Balance,returns
Date,,,,,,,,,,,,,,,,,,,,,
2019-05-06,7865.00,7865.00,7668.75,7803.50,7803.50,91921,7779.08,7693.17,7889.42,7582.83,...,7496.92,8085.67,NaN,NaN,196.25,-61.50,0.0,0.00,10000.00,0.0
2019-05-07,7745.25,7804.00,7581.50,7674.00,7674.00,159356,7686.50,7569.00,7791.50,7464.00,...,7346.50,8014.00,NaN,NaN,222.50,-71.25,0.0,0.00,10000.00,0.0
2019-05-08,7668.50,7697.75,7600.75,7647.50,7647.50,174381,7648.67,7599.58,7696.58,7551.67,...,7502.58,7793.58,NaN,NaN,97.00,-21.00,0.0,0.00,10000.00,0.0
2019-05-09,7640.00,7644.25,7481.25,7595.25,7595.25,181633,7573.58,7502.92,7665.92,7410.58,...,7339.92,7828.92,NaN,NaN,163.00,-44.75,0.0,0.00,10000.00,0.0
2019-05-10,7592.00,7653.50,7435.50,7610.25,7610.25,165887,7566.42,7479.33,7697.33,7348.42,...,7261.33,7915.33,NaN,NaN,218.00,18.25,0.0,0.00,10000.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-16,20625.00,20684.00,20464.00,20598.00,20598.00,1426846,20582.00,20480.00,20700.00,20362.00,...,20260.00,20920.00,20567.30,NaN,220.00,-27.00,0.0,180066.86,190066.86,0.0
2024-07-17,20592.00,20597.75,19965.75,19997.50,19997.50,1779691,20187.00,19776.25,20408.25,19555.00,...,19144.25,21040.25,20541.53,NaN,632.00,-594.50,0.0,180066.86,190066.86,0.0
2024-07-18,20024.25,20162.75,19758.75,19899.25,19899.25,1993373,19940.25,19717.75,20121.75,19536.25,...,19313.75,20525.75,20490.30,NaN,404.00,-125.00,0.0,180066.86,190066.86,0.0


In [7]:
benchmark = qs.utils.download_returns('MNQ=F') # note ticker format 
#qs.reports.full(returns_series , benchmark)
#qs.reports.full(benchmark)
# Html Tearsheet
# qs.reports.html(returns_series , benchmark=benchmark)

[*********************100%%**********************]  1 of 1 completed
